In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import warnings
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [3]:
company = pd.read_csv('/Users/diadld3naver.com/Downloads/data.csv')

In [4]:
company.columns = company.columns.str.replace(' ','')

In [5]:
company["NetIncomeFlag"].value_counts()

1    6819
Name: NetIncomeFlag, dtype: int64

In [6]:
company=company.drop('NetIncomeFlag',axis=1)

In [7]:
company.head()



,Bankrupt?,ROA(C)beforeinterestanddepreciationbeforeinterest,ROA(A)beforeinterestand%aftertax,ROA(B)beforeinterestanddepreciationaftertax,OperatingGrossMargin,RealizedSalesGrossMargin,OperatingProfitRate,Pre-taxnetInterestRate,After-taxnetInterestRate,Non-industryincomeandexpenditure/revenue,Continuousinterestrate(aftertax),OperatingExpenseRate,Researchanddevelopmentexpenserate,Cashflowrate,Interest-bearingdebtinterestrate,Taxrate(A),NetValuePerShare(B),NetValuePerShare(A),NetValuePerShare(C),PersistentEPSintheLastFourSeasons,CashFlowPerShare,RevenuePerShare(Yuan¥),OperatingProfitPerShare(Yuan¥),PerShareNetprofitbeforetax(Yuan¥),RealizedSalesGrossProfitGrowthRate,OperatingProfitGrowthRate,After-taxNetProfitGrowthRate,RegularNetProfitGrowthRate,ContinuousNetProfitGrowthRate,TotalAssetGrowthRate,NetValueGrowthRate,TotalAssetReturnGrowthRateRatio,CashReinvestment%,CurrentRatio,QuickRatio,InterestExpenseRatio,Totaldebt/Totalnetworth,Debtratio%,Networth/Assets,Long-termfundsuitabilityratio(A),Borrowingdependency,Contingentliabilities/Networth,Operatingprofit/Paid-incapital,Netprofitbeforetax/Paid-incapital,Inventoryandaccountsreceivable/Netvalue,TotalAssetTurnover,AccountsReceivableTurnover,AverageCollectionDays,InventoryTurnoverRate(times),FixedAssetsTurnoverFrequency,NetWorthTurnoverRate(times),Revenueperperson,Operatingprofitperperson,Allocationrateperperson,WorkingCapitaltoTotalAssets,QuickAssets/TotalAssets,CurrentAssets/TotalAssets,Cash/TotalAssets,QuickAssets/CurrentLiability,Cash/CurrentLiability,CurrentLiabilitytoAssets,OperatingFundstoLiability,Inventory/WorkingCapital,Inventory/CurrentLiability,CurrentLiabilities/Liability,WorkingCapital/Equity,CurrentLiabilities/Equity,Long-termLiabilitytoCurrentAssets,RetainedEarningstoTotalAssets,Totalincome/Totalexpense,Totalexpense/Assets,CurrentAssetTurnoverRate,QuickAssetTurnoverRate,WorkingcapitcalTurnoverRate,CashTurnoverRate,CashFlowtoSales,FixedAssetstoAssets,CurrentLiabilitytoLiability,CurrentLiabilitytoEquity,EquitytoLong-termLiability,CashFlowtoTotalAssets,CashFlowtoLiability,CFOtoAssets,CashFlowtoEquity,CurrentLiabilitytoCurrentAssets,Liability-AssetsFlag,NetIncometoTotalAssets,TotalassetstoGNPprice,No-creditInterval,GrossProfittoSales,NetIncometoStockholder'sEquity,LiabilitytoEquity,DegreeofFinancialLeverage(DFL),InterestCoverageRatio(InterestexpensetoEBIT),EquitytoLiability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,0.780985,1.256969e-04,0.0,0.458143,0.000725,0.0,0.147950,0.147950,0.147950,0.169141,0.311664,0.017560,0.095921,0.138736,0.022102,0.848195,0.688979,0.688979,0.217535,4.980000e+09,0.000327,0.263100,0.363725,0.002259,0.001208,0.629951,0.021266,0.207576,0.792424,0.005024,0.390284,0.006479,0.095885,0.137757,0.398036,0.086957,0.001814,0.003487,1.820926e-04,1.165007e-04,0.032903,0.034164,0.392913,0.037135,0.672775,0.166673,0.190643,0.004094,0.001997,1.473360e-04,0.147308,0.334015,0.276920,0.001036,0.676269,0.721275,0.339077,0.025592,0.903225,0.002022,0.064856,7.010000e+08,6.550000e+09,0.593831,4.580000e+08,0.671568,0.424206,0.676269,0.339077,0.126549,0.637555,0.458609,0.520382,0.312905,0.118250,0,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,0.781506,2.897851e-04,0.0,0.461867,0.000647,0.0,0.182251,0.182251,0.182251,0.208944,0.318137,0.021144,0.093722,0.169918,0.022080,0.848088,0.689693,0.689702,0.217620,6.110000e+09,0.000443,0.264516,0.376709,0.006016,0.004039,0.635172,0.012502,0.171176,0.828824,0.005059,0.376760,0.005835,0.093743,0.168962,0.397725,0.064468,0.001286,0.004917,9.360000e+09,7.190000e+08,0.025484,0.006889,0.391590,0.012335,0.751111,0.127236,0.182419,0.014948,0.004136,1.383910e-03,0.056963,0.341106,0.289642,0.005210,0.308589,0.731975,0.329740,0.023947,0.931065,0.002226,0.025516,1.065198e-04,7.700000e+09,0.593916,2.490000e+09,0.671570,0.468828,0.308589,0.329740,0.120916,0.641100,0.459001,0.567101

In [75]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
over = SMOTE()
SDS=StandardScaler()

X = company.drop('Bankrupt?',axis=1)
y = company['Bankrupt?']
X,y = over.fit_resample(X,y)

In [76]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)


In [77]:
SDS.fit(X_train)
SDS_train=SDS.transform(X_train)
SDS_test=SDS.transform(X_test)

In [109]:
from sklearn.decomposition import PCA

pca = PCA(n_components=60)
X_train = pca.fit_transform(SDS_train)
X_test = pca.transform(SDS_test)

In [110]:
X_train.shape, X_test.shape

((9898, 60), (3300, 60))

In [111]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='lbfgs',max_iter=1000).fit(X_train,y_train)
logreg.fit(X_train,y_train)

y_log=logreg.predict(X_test)
kfold = KFold(n_splits=10, shuffle = True, random_state=0)
scores = cross_val_score(logreg ,X,y ,cv=kfold)


print("훈련 세트 점수: {:.3f}".format(logreg.score(X_train,y_train)))
print("테스트 세트 점수: {:.3f}\n".format(logreg.score(X_test,y_test)))

print("Confusion Matrix\n",confusion_matrix(y_log,y_test))
print("\nClassification Report\n\n",classification_report(y_log,y_test))

print('교차 검증별 정확도:\n',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

훈련 세트 점수: 0.906
테스트 세트 점수: 0.903

Confusion Matrix
 [[1479  133]
 [ 187 1501]]

Classification Report

               precision    recall  f1-score   support

           0       0.89      0.92      0.90      1612
           1       0.92      0.89      0.90      1688

    accuracy                           0.90      3300
   macro avg       0.90      0.90      0.90      3300
weighted avg       0.90      0.90      0.90      3300

교차 검증별 정확도:
 [0.6023 0.5939 0.6061 0.6098 0.622  0.6485 0.6212 0.6167 0.6035 0.5747]
평균 검증 정확도: 0.6099


In [112]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=4, random_state=0)
tree.fit(X_train, y_train)

y_de=tree.predict(X_test)
kfold = KFold(n_splits=10, shuffle = True, random_state=0)
scores = cross_val_score(tree ,X,y ,cv=kfold)


print("훈련 세트 점수: {:.3f}".format(tree.score(X_train,y_train)))
print("테스트 세트 점수: {:.3f}\n".format(tree.score(X_test,y_test)))

print("Confusion Matrix\n",confusion_matrix(y_de,y_test))
print("\nClassification Report\n\n",classification_report(y_de,y_test))

print('교차 검증별 정확도:\n',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

훈련 세트 점수: 0.894
테스트 세트 점수: 0.883

Confusion Matrix
 [[1423  142]
 [ 243 1492]]

Classification Report

               precision    recall  f1-score   support

           0       0.85      0.91      0.88      1565
           1       0.91      0.86      0.89      1735

    accuracy                           0.88      3300
   macro avg       0.88      0.88      0.88      3300
weighted avg       0.89      0.88      0.88      3300

교차 검증별 정확도:
 [0.9083 0.8909 0.9197 0.8902 0.9045 0.8924 0.9023 0.9129 0.9181 0.8878]
평균 검증 정확도: 0.9027


In [113]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=3, random_state=0)
forest.fit(X_train, y_train)

y_fo=forest.predict(X_test)
kfold = KFold(n_splits=10, shuffle = True, random_state=0)
scores = cross_val_score(forest ,X,y ,cv=kfold)


print("훈련 세트 점수: {:.3f}".format(forest.score(X_train,y_train)))
print("테스트 세트 점수: {:.3f}\n".format(forest.score(X_test,y_test)))

print("Confusion Matrix\n",confusion_matrix(y_fo,y_test))
print("\nClassification Report\n\n",classification_report(y_fo,y_test))

print('교차 검증별 정확도:\n',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

훈련 세트 점수: 0.990
테스트 세트 점수: 0.947

Confusion Matrix
 [[1539   48]
 [ 127 1586]]

Classification Report

               precision    recall  f1-score   support

           0       0.92      0.97      0.95      1587
           1       0.97      0.93      0.95      1713

    accuracy                           0.95      3300
   macro avg       0.95      0.95      0.95      3300
weighted avg       0.95      0.95      0.95      3300

교차 검증별 정확도:
 [0.9659 0.9583 0.9659 0.9667 0.9568 0.9644 0.9735 0.9652 0.9674 0.9492]
평균 검증 정확도: 0.9633


In [108]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=10, random_state=0)
forest.fit(X_train, y_train)

y_fo=forest.predict(X_test)
kfold = KFold(n_splits=10, shuffle = True, random_state=0)
scores = cross_val_score(forest ,X,y ,cv=kfold)


print("훈련 세트 점수: {:.3f}".format(forest.score(X_train,y_train)))
print("테스트 세트 점수: {:.3f}\n".format(forest.score(X_test,y_test)))

print("Confusion Matrix\n",confusion_matrix(y_fo,y_test))
print("\nClassification Report\n\n",classification_report(y_fo,y_test))

print('교차 검증별 정확도:\n',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

훈련 세트 점수: 0.999
테스트 세트 점수: 0.971

Confusion Matrix
 [[1601   30]
 [  65 1604]]

Classification Report

               precision    recall  f1-score   support

           0       0.96      0.98      0.97      1631
           1       0.98      0.96      0.97      1669

    accuracy                           0.97      3300
   macro avg       0.97      0.97      0.97      3300
weighted avg       0.97      0.97      0.97      3300

교차 검증별 정확도:
 [0.9773 0.978  0.9803 0.972  0.9811 0.9795 0.9864 0.9818 0.981  0.9704]
평균 검증 정확도: 0.9788
